In [1]:
import pandas as pd 
import json

from datetime import datetime
from loguru import logger
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()
import os 

In [2]:
empresa = os.getenv('empresa')
company_id = os.getenv('cnpj')

In [3]:
arquivo = f"/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/balanco/{empresa}.json"
with open(arquivo, "r", encoding="utf-8") as f:
    dados = json.load(f)

In [4]:
empresa = dados['data']

In [5]:
def ativo_id():
    """
    Retorna o próximo ID para a tabela ativo_circulante.ativo_total.
    O ID é gerado com base no maior valor já existente na tabela.
    Se a tabela estiver vazia, o ID começará em 1.
    """
    df = pd.read_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativo_circulante.csv')
    if df.empty:
        return 1
    else:
        return df['ativo_id'].max() + 1

def inserir_ativo(base, lista_prefixos, lista_sufixos, lista_valores, ID):
    # con = PostgresConnector()
    for nome, descricao, valor in zip(lista_prefixos, lista_sufixos, lista_valores):
        try:
            if base == "Aplicações Financeiras":
                temp = pd.DataFrame(columns=['codigo','nome','descricao','valor'])
                temp['ativo_id'] = [ID]
                temp['nome'] = [nome]
                temp['descricao'] = [descricao]
                temp['valor'] = [valor]
                df_caixa = pd.read_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/aplicacao_financeira_descritivo.csv')
                df_caixa = pd.concat([df_caixa, temp], ignore_index=True)
                df_caixa.to_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/aplicacao_financeira_descritivo.csv',index=False)

            elif base == "Contas a Receber":
                temp = pd.DataFrame(columns=['codigo','nome','descricao','valor'])
                temp['ativo_id'] = [ID]
                temp['nome'] = [nome]
                temp['descricao'] = [descricao]
                temp['valor'] = [valor]
                df_aplicao_financeira = pd.read_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/contas_receber_descritivo.csv')
                df_aplicao_financeira = pd.concat([df_aplicao_financeira, temp], ignore_index=True)
                df_aplicao_financeira.to_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/contas_receber_descritivo.csv',index=False)

            elif base == "Tributos a Recuperar":
                temp = pd.DataFrame(columns=['codigo','nome','descricao','valor'])
                temp['ativo_id'] = [ID]
                temp['nome'] = [nome]
                temp['descricao'] = [descricao]
                temp['valor'] = [valor]
                df_contas_receber = pd.read_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/tributos_descritivo.csv')
                df_contas_receber = pd.concat([df_contas_receber, temp], ignore_index=True)
                df_contas_receber.to_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/tributos_descritivo.csv',index=False)
            
            elif base == "Outros Ativos Circulantes":
                temp = pd.DataFrame(columns=['codigo','nome','descricao','valor'])
                temp['ativo_id'] = [ID]
                temp['nome'] = [nome]
                temp['descricao'] = [descricao]
                temp['valor'] = [valor]
                df_estoque = pd.read_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/outros_descritivo.csv')
                df_estoque = pd.concat([df_estoque, temp], ignore_index=True)
                df_estoque.to_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/outros_descritivo.csv',index=False)

        except Exception as e:
            logger.error("Erro na função inserir_ativo")
            logger.error(f"Erro ao processar item ({nome}, {descricao}, {valor}): {e}")
            break
    # con.close()

def separar_classes(procurar, resultado, ID):
    lista_prefixos = []
    lista_sufixos = []
    lista_valores = []

    caminhos_pos_procurar = []

    # Coletar todos os caminhos abaixo do "procurar"
    for chave in resultado:
        partes = chave.split(" | ")
        indices_procurar = [i for i, p in enumerate(partes) if p == procurar]
        idx = next((i for i in indices_procurar if i + 1 <= len(partes)), None)
        if idx is not None:
            caminho = partes[idx + 1:]
            caminhos_pos_procurar.append(caminho)

    # Determinar quais caminhos são folhas
    folhas = []
    for caminho in caminhos_pos_procurar:
        is_folha = not any(
            outro != caminho and outro[:len(caminho)] == caminho
            for outro in caminhos_pos_procurar
        )
        if is_folha:
            folhas.append(caminho)

    # Agora, coletar os valores somente dos nós folha
    for chave, valor in resultado.items():
        partes = chave.split(" | ")
        indices_procurar = [i for i, p in enumerate(partes) if p == procurar]
        idx = next((i for i in indices_procurar if i + 1 <= len(partes)), None)
        if idx is None:
            continue

        caminho = partes[idx + 1:]

        if caminho in folhas:
            if len(caminho) == 0:
                lista_prefixos.append(procurar)
                lista_sufixos.append(procurar)
            elif len(caminho) == 1:
                lista_prefixos.append(caminho[0])
                lista_sufixos.append(caminho[0])
            elif len(caminho) > 1:
                lista_prefixos.append(caminho[0])
                lista_sufixos.append(caminho[1])
            lista_valores.append(valor)

    inserir_ativo(procurar, lista_prefixos, lista_sufixos, lista_valores, ID)

def circulante( id=None, COMPANIA_ID=None, dados=None):

    valor_procurado = "Ativo Total | Ativo Circulante"
    lista  = ["Ativo Total | Ativo Circulante | Caixa e Equivalentes de Caixa",
                "Ativo Total | Ativo Circulante | Aplicações Financeiras",
                "Ativo Total | Ativo Circulante | Contas a Receber",
                "Ativo Total | Ativo Circulante | Estoques",
                "Ativo Total | Ativo Circulante | Ativos Biológicos",
                "Ativo Total | Ativo Circulante | Tributos a Recuperar",
                "Ativo Total | Ativo Circulante | Despesas Antecipadas",
                "Ativo Total | Ativo Circulante | Outros Ativos Circulantes"
                ]


    resultado_atual = {}

    for chave, valor in dados.items():
        if valor_procurado in chave:
            resultado_atual[chave] = valor

    for l in lista:
        valor_procurado = l
        partes = valor_procurado.split(" | ")
        if len(partes) >= 3:
            procurar = partes[2]

        resultado = {}

        for chave, valor in resultado_atual.items():
            if l in chave:
                resultado[chave] = valor

        separar_classes(procurar, resultado, id)

def inserir_resumo(dados,id):
    caixa = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/caixa.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Caixa e Equivalentes de Caixa"]]
    caixa = pd.concat([caixa,temp])
    caixa.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/caixa.csv",index=False)

    aplicacao_financeira = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/aplicacao_financeira.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Aplicações Financeiras"]]
    aplicacao_financeira = pd.concat([aplicacao_financeira,temp])
    aplicacao_financeira.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/aplicacao_financeira.csv",index=False)

    contas = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/contas_receber.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Contas a Receber"]]
    contas = pd.concat([contas,temp])
    contas.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/contas_receber.csv",index=False)

    estoque = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/estoque.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Estoques"]]
    estoque = pd.concat([estoque,temp])
    estoque.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/estoque.csv",index=False)

    biologico = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativos_biologicos.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Ativos Biológicos"]]
    biologico = pd.concat([biologico,temp])
    biologico.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativos_biologicos.csv",index=False)

    tributos = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/tributos.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Tributos a Recuperar"]]
    tributos = pd.concat([tributos,temp])
    tributos.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/tributos.csv",index=False)

    despesas = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/despesas.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Despesas Antecipadas"]]
    despesas = pd.concat([despesas,temp])
    despesas.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/despesas.csv",index=False)

    outros = pd.read_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/outros.csv")
    temp = pd.DataFrame(columns=['ativo_id','valor'])
    temp['ativo_id'] = [id]
    temp['valor'] = [dados['data']["Ativo Total | Ativo Circulante | Outros Ativos Circulantes"]]
    outros = pd.concat([outros,temp])
    outros.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/outros.csv",index=False)

In [6]:
ativo_circulante = pd.DataFrame(columns=['ativo_id','company_id','reference_date','start_date','end_date','publish_date','retrieval_date','valor'])
caixa = pd.DataFrame(columns=['ativo_id','valor'])
aplicacao_financeira = pd.DataFrame(columns=['ativo_id','valor'])
aplicacao_financeira_descritivo = pd.DataFrame(columns=['ativo_id','nome','descricao','valor'])
contas_receber = pd.DataFrame(columns=['ativo_id','valor'])
contas_receber_descritivo = pd.DataFrame(columns=['ativo_id','nome','descricao','valor'])
estoque = pd.DataFrame(columns=['ativo_id','valor'])
ativos_biologicos = pd.DataFrame(columns=['ativo_id','valor'])
tributos = pd.DataFrame(columns=['ativo_id','valor'])
tributos_descritivo = pd.DataFrame(columns=['ativo_id','nome','descricao','valor'])
despesas = pd.DataFrame(columns=['ativo_id','valor'])
outros = pd.DataFrame(columns=['ativo_id','valor'])
outros_descritivo = pd.DataFrame(columns=['ativo_id','nome','descricao','valor'])

ativo_circulante.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativo_circulante.csv",index=False)
caixa.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/caixa.csv",index=False)
aplicacao_financeira.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/aplicacao_financeira.csv",index=False)
aplicacao_financeira_descritivo.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/aplicacao_financeira_descritivo.csv",index=False)
contas_receber.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/contas_receber.csv",index=False)
contas_receber_descritivo.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/contas_receber_descritivo.csv",index=False)
estoque.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/estoque.csv",index=False)
ativos_biologicos.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativos_biologicos.csv",index=False)
despesas.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/despesas.csv",index=False)
tributos.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/tributos.csv",index=False)
tributos_descritivo.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/tributos_descritivo.csv",index=False)
outros.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/outros.csv",index=False)
outros_descritivo.to_csv("/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/outros_descritivo.csv",index=False)


In [7]:
data = [] 

empresa = dados['data']

for emp in empresa:
    if emp['report_type']['acronym'] == 'BPA' and emp['reference_date'] not in data:

        data.append(emp['reference_date'])
        try: 
            id = ativo_id()
            inserir_resumo(emp, id)
            temp = pd.DataFrame(columns=['company_id','reference_date','start_date','end_date','publish_date','retrieval_date'])
            temp['ativo_id'] = [id]
            temp['company_id'] = [company_id]        
            temp['reference_date'] = [emp['reference_date']]
            temp['start_date'] = [emp['start_date']]
            temp['end_date'] = [emp['end_date']]
            temp['publish_date'] = [emp['publish_date']]
            temp['retrieval_date'] = [emp['retrieval_date']]
            temp['valor'] = [emp['data']["Ativo Total | Ativo Circulante"]]
            
            df_ativo_valor = pd.read_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativo_circulante.csv')
            df_ativo_valor = pd.concat([df_ativo_valor, temp], ignore_index=True)
            df_ativo_valor.to_csv('/home/rafael/00-Projetos/Renda_Investe/renda_investe-banco_de_dados/data/bd/ativo_circulante/ativo_circulante.csv',index=False)
            
            circulante(id, company_id, emp['data'])
            
        except:
            logger.error(f"Falha na data {emp['reference_date']} da empresa {company_id}")